In [1]:
!pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import shlex
import timeit


# Read the csv file
data=open("100k.txt",'r')

In [2]:
count=0
tabels={}
for x in data:
    parameters = shlex.split(x, posix=False)
    subject = parameters[0].split(":")
    property = parameters[1].split(":")
    object = parameters[2].split(":")
    count=count+1
    temp = []
    # print(parameters)
    if(property[1] in tabels):
        temp = tabels[property[1]]
    temp.append((subject[1], object[1] if len(object)>1 else object[0]))
    tabels[property[1]]= temp


# dict(sorted(tabels['follows']))
file = open("TablesFormattedData.txt", "w")
file.write(str(tabels))

file.close()
data.close()

In [3]:
userLookupTable=tabels['userId']
encodeUserIdDict ={}
decodeUserIdDict = {}

for x in userLookupTable:
    encodeUserIdDict[x[0]]=int(x[1].split('"')[1])
    decodeUserIdDict[int(x[1].split('"')[1])] = x[0]
OldfollowTabel = tabels['follows']
followTabel =[]
for x in OldfollowTabel:
    followTabel.append((x[0],x[1]))
    
OldfriendOfTabel = tabels['friendOf']
friendOfTabel =[]
for x in OldfriendOfTabel:
    friendOfTabel.append((x[0],x[1]))
OldlikesTabel = tabels['likes']
likesTabel =[]
for x in OldlikesTabel:
    likesTabel.append((x[0],x[1]))
reviewTabel = tabels['hasReview']

In [4]:
'''

'''
followTabelMergeJoin=followTabel
friendOfTabelMergeJoin= friendOfTabel
likesTabelMergeJoin = likesTabel
reviewTabelMergeJoin = reviewTabel
# Sort Merge Join


In [5]:
count = 0


def getValue(key,x):
    return int(x.split(key)[1])

def mergeFunction(table1, table2, key):
    result =[]
    table1lPtr=0
    table2Ptr=0
    ind=0
    initialStart=-1
    count=0
    
    while (table1lPtr < len(table1) and table2Ptr < len(table2)):
        table1Content = table1[table1lPtr]
        table1Subject = list(table1Content[0:len(table1Content)-1])
        table1Object = table1Content[-1]

        table2Content = table2[table2Ptr]
        table2Subject = table2Content[0]
        table2Object = table2Content[1]
        if(str(table1Object).lower() == str(table2Subject).lower()):
            result.append(tuple(table1Subject+[table1Object]+[table2Object]))
            if(ind==0):
                initialStart=table2Ptr
                ind=1
            table2Ptr = table2Ptr + 1
        elif (getValue(key,table1Object) < getValue(key,table2Subject)):
            table1lPtr = table1lPtr + 1
            if(str(table1[table1lPtr][-1]).lower()==str(table1[table1lPtr-1][-1]).lower() and ind == 1):
                table2Ptr = initialStart
            ind=0
        else:
            table2Ptr = table2Ptr + 1
            
            
    while(table1lPtr < len(table1)):
        
        table1lPtr = table1lPtr + 1
        if(table1lPtr < len(table1)):
            table1Content = table1[table1lPtr]
            table1Subject = list(table1Content[0:len(table1Content)-1])
            table1Object = table1Content[-1]

            if(str(table1[table1lPtr][-1]).lower()==str(table1[table1lPtr-1][-1]).lower()):
                table2Ptr = initialStart

                table2Content = table2[table2Ptr]
                table2Subject = table2Content[0]
                table2Object = table2Content[1]

                while(str(table1Object).lower() == str(table2Subject).lower()):

                    result.append(tuple(table1Subject+[table1Object]+[table2Object]))

                    table2Ptr = table2Ptr + 1
                    if(table2Ptr < len(table2)):
                        table2Content = table2[table2Ptr]
                        table2Subject = table2Content[0]
                        table2Object = table2Content[1]
                    else:
                        break
        else:
            break        
            
    return result

In [6]:
# 1st Step Sorting
start = timeit.default_timer()
followTabelMergeJoin=sorted(followTabelMergeJoin, key=lambda element: (int(element[-1].split('User')[1])))
friendOfTabelMergeJoin = sorted(friendOfTabelMergeJoin, key =lambda element: (int(element[0].split('User')[1]),int(element[1].split('User')[1])))

likesTabelMergeJoin = sorted(likesTabelMergeJoin,key = lambda element: (int(element[0].split('User')[1]),int(element[1].split('Product')[1])))
reviewTabelMergeJoin = sorted(reviewTabelMergeJoin, key=lambda element: (int(element[0].split('Product')[1]), int(element[1].split('Review')[1])))


## Merge Step for followTable and friendOfTabel
key = 'User'
result = mergeFunction(followTabelMergeJoin,friendOfTabelMergeJoin,key)
result = sorted(result, key =lambda element: (int(element[-1].split(key)[1])))
# print(len(result))
### Merge Step for (friendOfTabel,followTable) and likesTabel
result = mergeFunction(result,likesTabelMergeJoin,key)
key = 'Product'
### Merge Step for (friendOfTabel,followTable,likesTabel) and reviewTabel
result = sorted(result, key =lambda element: (int(element[-1].split(key)[1])))
result = mergeFunction(result,reviewTabelMergeJoin,key)
key = 'User'

stop = timeit.default_timer()

print('Time: ', stop - start)  

### Just sorting the result Extra etep
result = sorted(result, key =lambda element: (int(element[0].split(key)[1]),int(element[1].split(key)[1]),int(element[2].split(key)[1]),int(element[3].split('Product')[1]),int(element[4].split('Review')[1])))

Time:  21.816362200001095


In [7]:
print(len(result))
file = open("SortMergeJoin.txt", "w")
file.write(str(result))

file.close()

11415461


In [8]:
followTabelHashJoin = followTabel
friendOfTabelHashJoin = friendOfTabel
likesTabelHashJoin = likesTabel
reviewTabelHashJoin = reviewTabel

In [9]:
import hashlib
def makeHashFunction(table2):
    hashDic = {}
    for x in table2:
        grpList=set()
        hashvalue = hashlib.md5(x[0].encode('utf-8')).hexdigest()
        
        if hashlib.md5(x[0].encode('utf-8')).hexdigest() in hashDic:
            grpList=hashDic[hashlib.md5(x[0].encode('utf-8')).hexdigest()]
        grpList.add(x[1])
        hashDic[hashlib.md5(x[0].encode('utf-8')).hexdigest()] = grpList
    return hashDic

def joinUsingHash(tabel1,hashFunction):
    result = []
    for table1Content in tabel1:
        tabel1Subject = list(table1Content[0:len(table1Content)-1])
        tabel1Object = table1Content[-1]
        if (hashlib.md5(tabel1Object.encode('utf-8')).hexdigest() in hashFunction):
            totalHashedValues = hashFunction[hashlib.md5(tabel1Object.encode('utf-8')).hexdigest()]
            for values in totalHashedValues:
                result.append(tuple(tabel1Subject+[tabel1Object]+[values]))
    return result

In [10]:
start = timeit.default_timer()

hashTabelFriendOf = makeHashFunction(friendOfTabelHashJoin)
result = joinUsingHash(followTabelHashJoin,hashTabelFriendOf)
# print(len(result))
hashTabelLikes = makeHashFunction(likesTabelHashJoin)
result = joinUsingHash(result,hashTabelLikes)
hashTabelReview = makeHashFunction(reviewTabelHashJoin)
result = joinUsingHash(result,hashTabelReview)
stop = timeit.default_timer()

print('Time: ', stop - start)  

Time:  9.22100429999955


In [11]:
result = set(result)
print(len(result))
file = open("HashJoin.txt", "w")
file.write(str(result))

file.close()

11415461


In [20]:
followTabelHashJoin = followTabel
friendOfTabelHashJoin = friendOfTabel
likesTabelHashJoin = likesTabel
reviewTabelHashJoin = reviewTabel

In [21]:
def makeHashFunction(table2):
    hashDic = {}
    for x in table2:
        grpList=set()
        if hash(x[0]) in hashDic:
            grpList=hashDic[hash(x[0])]
        grpList.add(x[1])
        hashDic[hash(x[0])] = grpList
    return hashDic

def joinUsingHash(tabel1,hashFunction):
    result = []
    for table1Content in tabel1:
        tabel1Subject = list(table1Content[0:len(table1Content)-1])
        tabel1Object = table1Content[-1]
        if (hash(tabel1Object) in hashFunction):
            totalHashedValues = hashFunction[hash(tabel1Object)]
            for values in totalHashedValues:
                result.append(tuple(tabel1Subject+[tabel1Object]+[values]))
    return result

In [22]:
start = timeit.default_timer()

hashTabelFriendOf = makeHashFunction(friendOfTabelHashJoin)
# print(hashTabelFriendOf)
result = joinUsingHash(followTabelHashJoin,hashTabelFriendOf)
# print(len(result))
hashTabelLikes = makeHashFunction(likesTabelHashJoin)
result = joinUsingHash(result,hashTabelLikes)
hashTabelReview = makeHashFunction(reviewTabelHashJoin)
result = joinUsingHash(result,hashTabelReview)
stop = timeit.default_timer()

print('Time: ', stop - start)

Time:  5.894130899998345


In [23]:
result = set(result)
print(len(result))
file = open("ModifiedHashJoin.txt", "w")
file.write(str(result))

file.close()

11415461


In [24]:
'''
Second modified approach for 3rd part
'''

'\nSecond modified approach for 3rd part\n'

In [28]:
followTabelHashJoin = followTabel
friendOfTabelHashJoin = friendOfTabel
likesTabelHashJoin = likesTabel
reviewTabelHashJoin = reviewTabel

In [29]:
def makeHashFunctionModified(table2,pos):
    hashDic = {}
    for x in table2:
        grpList=set()
        if hash(x[pos]) in hashDic:
            grpList=hashDic[hash(x[pos])]
        if pos==0:
            grpList.add(x[1])
        else:
            grpList.add(x[0:])
        hashDic[hash(x[pos])] = grpList
    return hashDic

def joinUsingHashModified(tabel1,hashFunction):
    result = []
    for table1Keys,table1Items in tabel1.items():
        hashValue = table1Keys
        if (hashValue in hashFunction):
            for table1item in table1Items:
                tabel1Subject = list(table1item[0:len(table1Items)-1])
                tabel1Object = table1item[-1]
                totalHashedValues = hashFunction[hashValue]
                for values in totalHashedValues:
                    result.append(tuple(tabel1Subject+[tabel1Object]+[values]))
    return result

In [30]:
start = timeit.default_timer()

hashTabelFriendOf = makeHashFunctionModified(friendOfTabelHashJoin,0)
followTabelHashJoin = makeHashFunctionModified(followTabelHashJoin,-1)
result = joinUsingHashModified(followTabelHashJoin,hashTabelFriendOf)
# print(result[0])
hashTabelLikes = makeHashFunctionModified(likesTabelHashJoin,0)
result = makeHashFunctionModified(result,-1)
result = joinUsingHashModified(result,hashTabelLikes)


hashTabelReview = makeHashFunctionModified(reviewTabelHashJoin,0)
result = makeHashFunctionModified(result,-1)
result = joinUsingHashModified(result,hashTabelReview)
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  7.995889100000568


In [31]:
result = set(result)
print(len(result))
file = open("ModifiedDoubleHashedJoin.txt", "w")
file.write(str(result))

file.close()

11415461
